In [239]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import FixedEmbeddingComposite, EmbeddingComposite
from dwave_networkx.algorithms.max_cut import maximum_cut
from minorminer import find_embedding
import dwave.inspector
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx
import numpy as np
import os, re
import json
import ast

In [252]:
def read_dict_from_file(fname):
    """
    Parameters
    ----------
    fname : string
        Name of a file that contains a dict
    
    Returns
    -------
    d : dict
        Dictionary contained in the file
    """
    with open(fname, mode='r') as f:
        data = f.read()
    d = ast.literal_eval(data)
    return d

def is_complete(graph):
    """ 
    Parameters
    ----------
    graph : dict
        Form: {edge: weight, ...}
    
    Returns
    -------
    is_complete : bool
        1 --> graph is complete, 0 --> graph is not complete
    """
    num_nodes, num_edges = len(set( np.array(list( graph.keys() )).flatten() )), len(graph.keys())
    return num_edges == num_nodes * (num_nodes - 1) / 2

def normalize(edges):
    """ 
    Parameters
    ----------
    edges : dict
        Form: {edge: weight, ...}
    
    Returns
    -------
    normalized_edges : dict
        edges with normalized edge weights
    """
    max_weight = np.max(np.absolute(list(edges.values())))
    for edge in edges.keys():
        edges[edge] /= max_weight
    return edges

def save_results(response, outfolder):
    """ 
    Saves output of the annealing into 3 files ('embedding.txt', 'solution.txt', 'timing.txt') in outfolder.

    Parameters
    ----------
    response : D-Wave SampleSet
        Outpu of the annealing
    
    Returns
    -------
    None
    """
    
    def mkdir(dirname):
        try:
            os.mkdir(dirname)
        except FileExistsError:
            pass

    mkdir(outfolder)
    print(os.path.join(os.getcwd(), outfolder, 'embedding.txt'))
    with open(os.path.join(outfolder, 'embedding.txt'), mode='w') as outfile:
        outfile.write(json.dumps(response.info['embedding_context']['embedding']))
    with open(os.path.join(outfolder, 'solution.txt'), mode='w') as outfile:
        outfile.write(str(response.first)[7:])
    with open(os.path.join(outfolder, 'timing.txt'), mode='w') as outfile:
        outfile.write(json.dumps(response.info['timing']))

### Connect to a sampler

In [228]:
num_reads = 100
annealing_time = 20

for i, fname in enumerate(os.listdir(os.path.join(os.getcwd(), 'graphs_python_dictionary'))):

    J_dict = read_dict_from_file('graphs_python_dictionary/' + fname)

    h_dict = {}
    for node1, node2 in J_dict.keys():
        h_dict[node2] = 0.0
        h_dict[node1] = 0.0

    G = nx.Graph(J_dict.keys())

    sampler = DWaveSampler(profile='viderzen_protonmail')
    embedding = find_embedding(G, sampler.properties['couplers'], verbose=3)
    sampler = FixedEmbeddingComposite(sampler, embedding)
    response = sampler.sample_ising(h_dict, J_dict, num_reads=num_reads, annealing_time=annealing_time, chain_strength=2.3*np.max(np.absolute(list(J_dict.values()))))

    print(i + ': ' + fname + 'done.')

    save_results(response, os.path.join('results', fname[:-7]))